In [1]:
import pandas as pd
import re
import numpy as np
from numpy import array
import csv
import random

from sklearn.model_selection import train_test_split

# works for burp log
#HTTP_RE = re.compile(r"(GET|POST|TRACE|PUT|PATCH|DELETE)\s(.+?)\s+======================================================", re.MULTILINE | re.DOTALL)

HTTP_RE = re.compile(r"(GET|POST|TRACE|PUT|PATCH|DELETE)\s(.+?)\s+End - Id", re.MULTILINE | re.DOTALL)


def http_re(data):
    """
    Extracts HTTP requests from raw data string in special logging format.
    Logging format `ST@RT\n%(asctime)s %(levelname)-8s\n%(message)s\nEND`
    where `message` is a required HTTP request bytes.
    """
    return HTTP_RE.findall(data)


def get_requests_from_file(path):
    """
    Reads raw HTTP requests from given file.
    """
    with open(path, 'r') as f:
        file_data = f.read()
    requests = http_re(file_data)
    return requests

cisc_anomalousTraffic_test = get_requests_from_file("cisc_anomalousTraffic_test.txt")
cisc_normalTraffic_test = get_requests_from_file("cisc_normalTraffic_test.txt")
cisc_normalTraffic_train = get_requests_from_file("cisc_normalTraffic_test.txt")

files =[(cisc_anomalousTraffic_test,1),(cisc_normalTraffic_test,0),(cisc_normalTraffic_train,0)]
docs=[]
max_length=0

for file,label in files:
    for request in file:
        docs.append( (request[0]+" "+request[1],str(label)) )
        if len(request[1])>max_length:
            max_length=len(request[1])
            
df  = pd.DataFrame.from_records(docs,columns=['request','label'])
df['label'].apply(str)

train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
print("Training data length: "+str(len(train['request'])))
print("Validate data length: "+str(len(validate['request'])))
print("Test data length: "+str(len(test['request'])))

train.to_csv('../_train.csv',header=False,index=False,quoting=csv.QUOTE_ALL,columns=['label','request'])
validate.to_csv('../_validate.csv',header=False,index=False,quoting=csv.QUOTE_ALL,columns=['label','request'])
test.to_csv('../_test.csv',header=False,index=False,quoting=csv.QUOTE_ALL,columns=['label','request'])
print("MAX input length: "+str(max_length))



Training data length: 58239
Validate data length: 19413
Test data length: 19413
MAX input length: 1402
